In [1]:
path = 'data/'

In [2]:
import pandas as pd
import ast
from datasets import Dataset
from collections import Counter

train_df = pd.read_csv(path+"Labelled_Training_Data_Preprocessed.csv")
train_df = train_df[['comment', 'aspect_term', 'Sentiment']]
train_df = train_df.rename(columns={'comment': 'Sentence', 'aspect_term': 'Term', 'Sentiment': 'Sentiment'})
train_df = train_df.iloc[:3500]
# Assuming your DataFrame column is named 'Sentiment'
mapping_dict = {0: 'neutral', 1: 'positive', -1: 'negative'}

# Map values in the 'Sentiment' column using the mapping dictionary
train_df['Sentiment'] = train_df['Sentiment'].map(mapping_dict)





print("Polarity Counts:")
print(train_df.Sentiment.value_counts())

dataset = Dataset.from_pandas(train_df)
print("-"*10)
dataset

Polarity Counts:
Sentiment
neutral     1863
negative     895
positive     742
Name: count, dtype: int64
----------


Dataset({
    features: ['Sentence', 'Term', 'Sentiment'],
    num_rows: 3500
})

In [3]:
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
# huggingface hub model id
# model_id = "google/flan-t5-base"
model_id = "google/flan-t5-large"
# model_id = "shorthillsai/flan-t5-large-absa"
# model_id = "Shakhovak/flan-t5-large-absa-rest"
# model_id="google/flan-t5-xl"
# model_id = path+"t5 atpc"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto") #load_in_8bit=True, torch_dtype=torch.bfloat16,

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_id="google/flan-t5-large"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
# start_token = "[START]"
sep_token = "[NEXT]"
end_token = "[END]"

In [6]:
sep_token

'[NEXT]'

In [7]:
from random import randrange, seed

# Set a seed for reproducibility
seed_value = 12  # You can choose any seed value
seed(seed_value)

# Select three random indices from the dataset
selected_indices = [randrange(len(dataset)) for _ in range(3)]
# Format the three randomly chosen examples using selected_indices
examples = ""
for i in selected_indices:
    example = dataset[i]
    formatted_text = example['Sentence'].strip() + "\n"
    formatted_terms = "Aspect Term: "+(example['Term'].strip()) + "\n"
    formatted_terms += "Sentiment: " + example['Sentiment'].lower() + "\n"
    examples += formatted_text + formatted_terms + "\n"

def input_format(sample):
    # Combine the examples with the current sample
    formatted_input = f"""
{examples[:-1]}
{sample['Sentence'].strip()}
Aspect Term: {sample['Term'].strip()}
Sentiment: """ #{examples[:-1]}
    sample['input'] = formatted_input.lstrip()
    sample['output'] = sample['Sentiment'].lower()
    return sample

ran = randrange(len(dataset))
print(input_format(dataset[ran])['input'])
print(input_format(dataset[ran])['output'])

ashlee decides he is a lesbian trans must other lesbians accept his kkj as part of the deal or be condemned as transphobic ?
Aspect Term: trans
Sentiment: neutral

doesn t mean a thing imagine using this argument for section 377a .
Aspect Term: 377a
Sentiment: neutral

we don t know how many of the population opposes it , unlike 377a where we have ipos survey we have a survey showing that it s not a monolithic majority .
Aspect Term: 377a
Sentiment: neutral

i do see myself as being a moderate , but leaning towards affording the same rights .
Aspect Term: rights
Sentiment: 
positive


In [8]:
dataset = dataset.map(input_format)
dataset

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Dataset({
    features: ['Sentence', 'Term', 'Sentiment', 'input', 'output'],
    num_rows: 3500
})

In [9]:
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = dataset.map(lambda x: tokenizer(x["input"], truncation=True), batched=True, remove_columns=['Sentence', 'Term', 'Sentiment', 'input', 'output'])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 95))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = dataset.map(lambda x: tokenizer(x["output"], truncation=True), batched=True, remove_columns=['Sentence', 'Term', 'Sentiment', 'input', 'output'])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 100))
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Max source length: 223


Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Max target length: 2


In [10]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["input"]] #"For the following sentence, identify and output aspect terms with their sentiment polarities: " +

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True) #+20

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["output"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['Sentence', 'Term', 'Sentiment', 'input', 'output'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")

# save datasets to disk for later easy loading
# tokenized_dataset["train"].save_to_disk("data/train")
# tokenized_dataset["test"].save_to_disk("data/eval")


Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [11]:
from datasets import DatasetDict

# Suppose tokenized_dataset is your Dataset object.
# This is how you'd split it, with a 10% split for 'eval' and 90% for 'train'.
split_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=seed_value)

# Now, create a DatasetDict to keep your 'train' and 'eval' splits organized
final_datasets = DatasetDict({
    'train': split_datasets['train'],
    'eval': split_datasets['test']  # 'test' is the default name for the second split
})

In [12]:
final_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2800
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 700
    })
})

In [13]:
del(dataset)

In [14]:
import numpy as np
import ast
from sklearn.metrics import f1_score


def get_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    return f1

def extract_aspects_sentiments(decoded_output):
    sent = ['negative','neutral','positive']
    if decoded_output in sent:
      return decoded_output
    else:
      return "invalid"

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Check the structure of preds
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # preds = np.argmax(preds, axis=-1)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    # print(preds)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_preds = [extract_aspects_sentiments(line) for line in decoded_preds]
    decoded_labels = [extract_aspects_sentiments(line) for line in decoded_labels]

    # Calculate F1, precision, and recall
    f1 = get_metrics(decoded_labels, decoded_preds)

    # Prepare the result dictionary
    result = {"f1": f1}

    return result

In [15]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [16]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=128,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_kbit_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989059049678777


In [17]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import GenerationConfig

output_dir="lora-flan-t5-xl"

# generation_config= GenerationConfig(do_sample=False, max_new_tokens = 200)


# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	  per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=20,
    learning_rate=5e-5,
    # warmup_ratio=0.05,
    # weight_decay=0.1,
    # warmup_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    optim = 'adafactor',
    # lr_scheduler_type = 'linear',
    # save_total_limit=1,
    predict_with_generate=True,
    # learning_rate=1e-4, # higher learning rate
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    # logging_steps=50,
    seed=seed_value,
    generation_max_length = max_source_length+max_target_length+5,
    # generation_config = generation_config,
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=final_datasets['train'],
    eval_dataset=final_datasets['eval'],
    compute_metrics=compute_metrics,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


c:\Users\jeffr\anaconda3\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
# train model
trainer.train()

  0%|          | 0/5616 [00:00<?, ?it/s]

c:\Users\jeffr\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.5029, 'grad_norm': 5.726001262664795, 'learning_rate': 4.791666666666667e-05, 'epoch': 1.0}


  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 0.32466182112693787, 'eval_f1': 0.724978680452855, 'eval_runtime': 79.3885, 'eval_samples_per_second': 8.817, 'eval_steps_per_second': 0.743, 'epoch': 1.0}


...

c:\Users\jeffr\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.1666, 'grad_norm': 1.3683775663375854, 'learning_rate': 0.0, 'epoch': 24.0}


  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 0.32264062762260437, 'eval_f1': 0.7687774413592988, 'eval_runtime': 84.3378, 'eval_samples_per_second': 8.3, 'eval_steps_per_second': 0.7, 'epoch': 24.0}
{'train_runtime': 14384.097, 'train_samples_per_second': 4.672, 'train_steps_per_second': 0.39, 'train_loss': 0.2460821082449367, 'epoch': 24.0}


TrainOutput(global_step=5616, training_loss=0.2460821082449367, metrics={'train_runtime': 14384.097, 'train_samples_per_second': 4.672, 'train_steps_per_second': 0.39, 'train_loss': 0.2460821082449367, 'epoch': 24.0})

In [19]:
import os
name = "t5 first"
os.makedirs(name, exist_ok=True)
trainer.model.save_pretrained(name)
tokenizer.save_pretrained(name)

('t5 first\\tokenizer_config.json',
 't5 first\\special_tokens_map.json',
 't5 first\\spiece.model',
 't5 first\\added_tokens.json',
 't5 first\\tokenizer.json')

In [20]:
# torch.save(model.state_dict(), './t5 first/pytorch_model.bin')

In [21]:
import shutil

# Source directory path
source_dir = "t5 first"
# Destination directory path
destination_dir = "t5 atpc xl"

# Copy the entire folder
shutil.copytree(source_dir, destination_dir)

't5 atpc xl'

In [22]:
del(trainer)
del(model)
torch.cuda.empty_cache ()

In [23]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForSeq2SeqLM
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, set_seed,  BitsAndBytesConfig

In [24]:
path = ''

In [25]:
import os
model = AutoPeftModelForSeq2SeqLM.from_pretrained("t5 first", torch_dtype=torch.bfloat16, device_map="auto")  #, torch_dtype=torch.bfloat16
# model = model.merge_and_unload()
# os.makedirs("saved", exist_ok=True)
# # model.to(torch.float32)
# model.save_pretrained("saved", safe_serialization=True)
# del model

In [26]:
import pandas as pd
import ast
import torch

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5 first", trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [28]:
import pandas as pd
import ast
from collections import Counter

def inf_format(text, term):
    return f'''
{examples[:-1]}
{text}
Aspect Term: {term.strip()}
Sentiment: '''.lstrip() #{examples}

test_df = pd.read_csv("data/Labelled_Training_Data_Preprocessed.csv")
test_df = test_df[['comment', 'aspect_term', 'Sentiment']]
test_df = test_df.rename(columns={'comment': 'Sentence', 'aspect_term': 'Term', 'Sentiment': 'Sentiment'})
test_df = test_df.iloc[3500:]

# Assuming your DataFrame column is named 'Sentiment'
mapping_dict = {0: 'neutral', 1: 'positive', -1: 'negative'}

# Map values in the 'Sentiment' column using the mapping dictionary
test_df['Sentiment'] = test_df['Sentiment'].map(mapping_dict)


print("Polarity Counts in Test Data:")
print(test_df.Sentiment.value_counts())

Polarity Counts in Test Data:
Sentiment
neutral     788
negative    395
positive    317
Name: count, dtype: int64


In [29]:
test_df

,Sentence,Term,Sentiment
3500,singapore is after all a pioneer on trans righ...,trans,positive
3501,there s no regardless of sexual orientation in...,sexual orientation,neutral
3502,isn t it unfair that it s easier to be a strai...,homosexual,negative
3503,starting with this one small group speaking ou...,rights,positive
3504,just continue to pressure pap and they will gi...,same sex marriage,positive
...,...,...,...
4995,"also , if the 377a is repealed , the next step...",gay marriage,positive
4996,but i think he will not bother since 377a he a...,377a,negative
4997,"singles can purchase 2 room housing , which sh...",housing,neutral
4998,we don t know how many of the population oppos...,377a,neutral


In [30]:
from tqdm import tqdm
import torch

device = "cuda:0"
model.eval()

sent = ['negative','neutral','positive']

def extract_aspects_sentiments(decoded_output):
    decoded_output = decoded_output.strip()
    if decoded_output in sent:
      return decoded_output
    else:
      print("Invalid output: ", decoded_output)
      return "invalid"

def generate_output(inputs, max_tokens=50):
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs, max_new_tokens=max_tokens) #, temperature=0.1, top_p=1, do_sample= True, top_k=0
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    text = row['Sentence']
    term = row['Term']
    prompt = inf_format(text, term)
    inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    decoded_output = generate_output(inputs, max_tokens=200)
    # print(decoded_output)
    # break
    aspect_sentiments = extract_aspects_sentiments(decoded_output)
    test_df.at[index, 'predicted'] = str(aspect_sentiments)

print("Done")

  0%|          | 0/1500 [00:00<?, ?it/s]

100%|██████████| 1500/1500 [04:47<00:00,  5.22it/s]

Done


In [31]:
test_df['Sentiment'].value_counts()

Sentiment
neutral     788
negative    395
positive    317
Name: count, dtype: int64

In [32]:
test_df['Sentiment'] = test_df.apply(lambda row: row['Sentiment'].lower(), axis=1)

In [33]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(test_df['Sentiment'], test_df['predicted'], digits=2))

              precision    recall  f1-score   support

    negative       0.73      0.81      0.77       395
     neutral       0.86      0.78      0.82       788
    positive       0.72      0.81      0.76       317

    accuracy                           0.79      1500
   macro avg       0.77      0.80      0.78      1500
weighted avg       0.80      0.79      0.79      1500



In [34]:
import pandas as pd
import ast
from collections import Counter

def inf_format(text, term):
    return f'''
{examples[:-1]}
{text}
Aspect Term: {term.strip()}
Sentiment: '''.lstrip() #{examples}

label_df = pd.read_csv("data/Rest_of_data.csv")
label_df = label_df.rename(columns={'comment': 'Sentence', 'aspect_term': 'Term'})




In [35]:
from tqdm import tqdm
import torch

device = "cuda:0"
model.eval()

sent = ['negative','neutral','positive']

def extract_aspects_sentiments(decoded_output):
    decoded_output = decoded_output.strip()
    if decoded_output in sent:
      return decoded_output
    else:
      print("Invalid output: ", decoded_output)
      return "invalid"

def generate_output(inputs, max_tokens=50):
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs, max_new_tokens=max_tokens) #, temperature=0.1, top_p=1, do_sample= True, top_k=0
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output

for index, row in tqdm(label_df.iterrows(), total=label_df.shape[0]):
    text = row['Sentence']
    term = row['Term']
    prompt = inf_format(text, term)
    inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    decoded_output = generate_output(inputs, max_tokens=200)
    # print(decoded_output)
    # break
    aspect_sentiments = extract_aspects_sentiments(decoded_output)
    label_df.at[index, 'Sentiment'] = str(aspect_sentiments)

print("Done")

  0%|          | 0/20202 [00:00<?, ?it/s]

  0%|          | 78/20202 [00:15<1:05:38,  5.11it/s]


KeyboardInterrupt: 

In [ ]:
# Assuming your DataFrame column is named 'Sentiment'
mapping_dict = {'neutral': 0,'positive': 1, 'negative': -1}

# Map values in the 'Sentiment' column using the mapping dictionary
label_df['Sentiment'] = label_df['Sentiment'].map(mapping_dict)

label_df.to_csv("data/Rest_of_data_labelled.csv", index=False)

In [ ]:
inputs = tokenizer("What is the sentiment of the term Laptop in the sentence: Laptop is good but phone is bad", return_tensors="pt").input_ids.to(device)

decoded_output = generate_output(inputs, max_tokens=150)

print(decoded_output)

positive


In [ ]:
inputs = tokenizer("What is the full form of trans in Singapore?", return_tensors="pt").input_ids.to(device)

decoded_output = generate_output(inputs, max_tokens=150)

print(decoded_output)

transgender


In [ ]:
# from google.colab import runtime
# runtime.unassign()

In [ ]:
# from google.colab import runtime
# runtime.unassign()